# Protein Secondary Structure Prediction model test

In [22]:
import os, sys
sys.path.append('../src/')
import numpy as np
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.autograd import Variable
from data import *
from model import Net
from download_dataset import download_dataset
from sklearn.model_selection import KFold
from download_dataset import get_train
from utils import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
h = np.load('../data/result_201808052012/history_0.npy')

In [30]:
h

array([[2.47699230e+02, 1.15275610e+03, 3.67125725e-01],
       [2.37751225e+02, 1.09263770e+03, 4.70936479e-01]])

In [23]:
kf = KFold(10)

In [236]:
download_dataset()

Download CB513 dataset ...
Saved CB513 dataset in ../data/cb513.npz


In [37]:
D = LoadDataset(batch_size_train=64, batch_size_test=1)
idxs = np.arange(D.__len__())
kf = KFold(n_splits=10)

In [3]:
for i, idx in enumerate(kf.split(idxs)):
    train_loader, test_loader = D(idx)
    break

## device

In [38]:
use_cuda = not False and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## model

In [39]:
model = Net().load_state_dict(params).to(device)
loss = LossFunc()
optm = torch.optim.Adam(model.parameters())

AttributeError: 'NoneType' object has no attribute 'to'

In [6]:
model.train()
for data, target, seq_len in train_loader:
    data, target, seq_len = \
        data.to(device), target.to(device), seq_len.to(device)
    out = model(data)
#     l = Variable(loss(out, target, seq_len),  requires_grad = True)
#     l.backward()
    break

In [16]:
b = 0
for o, t, l in zip(out, target, seq_len):
    b += nn.CrossEntropyLoss()(o[:l], t[:l])

In [31]:
params = torch.load('../data/result_201808052012/model_1.pth')

In [33]:
params.keys()

odict_keys(['conv1.0.weight', 'conv1.0.bias', 'conv2.0.weight', 'conv2.0.bias', 'conv3.0.weight', 'conv3.0.bias', 'brnn.weight_ih_l0', 'brnn.weight_hh_l0', 'brnn.bias_ih_l0', 'brnn.bias_hh_l0', 'brnn.weight_ih_l0_reverse', 'brnn.weight_hh_l0_reverse', 'brnn.bias_ih_l0_reverse', 'brnn.bias_hh_l0_reverse', 'brnn.weight_ih_l1', 'brnn.weight_hh_l1', 'brnn.bias_ih_l1', 'brnn.bias_hh_l1', 'brnn.weight_ih_l1_reverse', 'brnn.weight_hh_l1_reverse', 'brnn.bias_ih_l1_reverse', 'brnn.bias_hh_l1_reverse', 'brnn.weight_ih_l2', 'brnn.weight_hh_l2', 'brnn.bias_ih_l2', 'brnn.bias_hh_l2', 'brnn.weight_ih_l2_reverse', 'brnn.weight_hh_l2_reverse', 'brnn.bias_ih_l2_reverse', 'brnn.bias_hh_l2_reverse', 'fc.0.weight', 'fc.0.bias', 'fc.2.weight', 'fc.2.bias'])

In [41]:
model = Net().to(device)
model.load_state_dict(params)

## data loader

In [458]:
model.train()
for data, target, seq_len in train_loader:
    data, target, seq_len = \
        data.to(device), target.to(device), seq_len.to(device)
    out = model(data)
    break